In [1]:
# Binary XOR operations
binary1 = 0b1011  # 1011 in binary
binary2 = 0b0100  # 0100 in binary
result_bin1 = bin(binary1 ^ binary2)

binary3 = 0b0110  # 0110 in binary
binary4 = 0b1110  # 1110 in binary
result_bin2 = bin(binary3 ^ binary4)

# Hexadecimal XOR operations
hex1 = 0x3F  # 3F in hexadecimal
hex2 = 0x42  # 42 in hexadecimal
result_hex1 = hex(hex1 ^ hex2)

hex3 = 0x2A  # 2A in hexadecimal
hex4 = 0x7B  # 7B in hexadecimal
result_hex2 = hex(hex3 ^ hex4)

# Output results
print("Binary Results:")
print(f"1011 XOR 0100 = {result_bin1}")
print(f"0110 XOR 1110 = {result_bin2}")

print("\nHexadecimal Results:")
print(f"3F XOR 42 = {result_hex1}")
print(f"2A XOR 7B = {result_hex2}")


Binary Results:
1011 XOR 0100 = 0b1111
0110 XOR 1110 = 0b1000

Hexadecimal Results:
3F XOR 42 = 0x7d
2A XOR 7B = 0x51


In [2]:
# Define the plaintext and the key
plaintext = [1, 0, 1, 1, 0, 1, 0, 1]
key = [0, 1, 0, 1, 1, 1, 0, 1]

# Encryption process (XOR plaintext with the key)
ciphertext = [p ^ k for p, k in zip(plaintext, key)]

# Decryption process (XOR ciphertext with the key to retrieve the plaintext)
decrypted_text = [c ^ k for c, k in zip(ciphertext, key)]

# Display the results
print("Plaintext:   ", plaintext)
print("Key:         ", key)
print("Ciphertext:  ", ciphertext)
print("Decrypted:   ", decrypted_text)


Plaintext:    [1, 0, 1, 1, 0, 1, 0, 1]
Key:          [0, 1, 0, 1, 1, 1, 0, 1]
Ciphertext:   [1, 1, 1, 0, 1, 0, 0, 0]
Decrypted:    [1, 0, 1, 1, 0, 1, 0, 1]


In [4]:
import os

# Function to XOR two byte arrays
def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

# Sample plaintext (convert string to bytes)
plaintext = b"welcome to CIS600!"  # You can choose any text for encryption

# Generate a pseudorandom keystream with the same length as the plaintext
keystream = os.urandom(len(plaintext))

# Encrypt the plaintext by XORing each byte with the keystream
ciphertext = xor_bytes(plaintext, keystream)

# Decrypt the ciphertext using the same keystream
decrypted_plaintext = xor_bytes(ciphertext, keystream)

# Print the results
print("Plaintext:           ", plaintext)
print("Generated Keystream: ", keystream)
print("Ciphertext:          ", ciphertext)
print("Decrypted Plaintext: ", decrypted_plaintext)


Plaintext:            b'welcome to CIS600!'
Generated Keystream:  b'\xf1\x81\xcfu`\xe8\x91\x92\x97?\x02\x1f\x9a4\x97V\x89\xe9'
Ciphertext:           b'\x86\xe4\xa3\x16\x0f\x85\xf4\xb2\xe3P"\\\xd3g\xa1f\xb9\xc8'
Decrypted Plaintext:  b'welcome to CIS600!'


In [6]:
# Function to XOR two byte arrays
def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

# Example messages (plaintexts)
P1 = b"Hello, World!"  # Plaintext 1 (known by attacker)
P2 = b"Secret Message"  # Plaintext 2 (unknown to attacker)

# Ensure both plaintexts are the same length for encryption
max_length = max(len(P1), len(P2))
P1 = P1.ljust(max_length)  # Padding P1 if necessary
P2 = P2.ljust(max_length)  # Padding P2 if necessary

# Generate a keystream (for both messages)
import os
keystream = os.urandom(max_length)  # Keystream length is same as the longest message

# Encrypt both plaintexts with the same keystream
C1 = xor_bytes(P1, keystream)  # Ciphertext 1
C2 = xor_bytes(P2, keystream)  # Ciphertext 2

# Attacker knows P1 and C1, so they compute the keystream
attacker_keystream = xor_bytes(P1, C1)

# Now, the attacker uses the keystream to decrypt C2 and retrieve P2
recovered_P2 = xor_bytes(C2, attacker_keystream).rstrip()  # Strip any padding if added

# Display the results and length
print("Known Plaintext P1:      ", P1)
print("Ciphertext C1:           ", C1)
print("Ciphertext C2:           ", C2)
print("Recovered Keystream:     ", attacker_keystream)
print("Recovered Plaintext P2:  ", recovered_P2)


Known Plaintext P1:       b'Hello, World! '
Ciphertext C1:            b'\x1f\n\xf6{\xa6\x93C\xf3\xba\x08\xf2Y8\xce'
Ciphertext C2:            b'\x04\n\xf9e\xac\xcbC\xe9\xb0\t\xed\\~\x8b'
Recovered Keystream:      b'Wo\x9a\x17\xc9\xbfc\xa4\xd5z\x9e=\x19\xee'
Recovered Plaintext P2:   b'Secret Message'


In [10]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

def encrypt_cbc(key, iv, plaintext):
    """Encrypts plaintext using AES in CBC mode."""
    # Create a cipher object with the specified key, mode, and backend
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Create an encryptor object from the cipher
    encryptor = cipher.encryptor()

    # Pad the plaintext to a multiple of the block size (16 bytes for AES)
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext) + padder.finalize()

    # Encrypt the padded data
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return ciphertext

def decrypt_cbc(key, iv, ciphertext):
    """Decrypts ciphertext using AES in CBC mode."""
    # Create a cipher object with the specified key, mode, and backend
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Create a decryptor object from the cipher
    decryptor = cipher.decryptor()

    # Decrypt the ciphertext
    plaintext_padded = decryptor.update(ciphertext) + decryptor.finalize()

    # Unpad the plaintext
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(plaintext_padded) + unpadder.finalize()

    return plaintext

# Example usage
key = os.urandom(32)  # Generate a random 32-byte key
iv = os.urandom(16)   # Generate a random 16-byte IV

plaintext_1 = b"This is a secret message."
ciphertext_1 = encrypt_cbc(key, iv, plaintext)
print("Ciphertext 1:", ciphertext_1)

plaintext_2 = b"Another secret message."
ciphertext_2 = encrypt_cbc(key, iv, plaintext_2)
print("Ciphertext 2:", ciphertext_2)

Ciphertext 1: b'7Kc\xae\xc2\xe4Gs\xc4u\x1e\x9a\x9d\x1dix\x8aw\xe0\xaf\xba\xfb\xd8\xcd\x1a\xb5K\xf8\xdf\x97\xf9\x8d'
Ciphertext 2: b'\x13w\xda\x92\x88\x1f\xdcJk\xa1X\xb5\xe8\x9c\xe7a\xa6\xcfI\xa8\x0b,\xbeg`\xe3\x12\x1aS\xf3\x84s'


In [11]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

def encrypt_cbc(key, iv, plaintext):
    """Encrypts plaintext using AES in CBC mode."""
    # Create a cipher object with the specified key, mode, and backend
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Create an encryptor object from the cipher
    encryptor = cipher.encryptor()

    # Pad the plaintext to a multiple of the block size (16 bytes for AES)
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext) + padder.finalize()

    # Encrypt the padded data
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return ciphertext

def decrypt_cbc(key, iv, ciphertext):
    """Decrypts ciphertext using AES in CBC mode."""
    # Create a cipher object with the specified key, mode, and backend
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Create a decryptor object from the cipher
    decryptor = cipher.decryptor()

    # Decrypt the ciphertext
    plaintext_padded = decryptor.update(ciphertext) + decryptor.finalize()

    # Unpad the plaintext
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(plaintext_padded) + unpadder.finalize()

    return plaintext

# Example usage
key = os.urandom(32)  # Generate a random 32-byte key
iv = os.urandom(16)   # Generate a random 16-byte IV

plaintext = b"This is a secret message."

# Encryption
print("key:", key)
print("IV", iv)

ciphertext = encrypt_cbc(key, iv, plaintext)
print("Ciphertext", ciphertext)


key: b'W\x87N\xcee\x84\xce\xf7L\xd6\xe7\xbaO\x00\xe0D\x16\xec<3\xd8axm\x8a_\x9c!\xe2\xea\xee|'
IV b'\xe3\xec\xa6A\x92(\xe5l\x7f\xbc\xb2\xc2\xc2\xdaTL'
Ciphertext b'\xcb\xca\xb0T/\xfd\x15)\x18\xb5[\x8aL8\x1a\xc2I\xcb\x7f\r\x1c(\xc3\xc8\x0eCc\xbd\x87E\xef\x08'
